In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Сессия 1. Подготовка датасета

Подготовка обучающей выборки: чтение данных, форматирование,дополнение фичей

In [2]:
# Импортирование данных
economy_df = pd.read_csv('../input/flight-price-prediction/economy.csv')
business_df = pd.read_csv('../input/flight-price-prediction/business.csv')

In [3]:
# Способ 1
# gen_values = np.vstack((business_df, economy_df))
# df_train = pd.DataFrame(gen_values, columns = business_df.columns)

# Способ 2
df_train = business_df.append(economy_df)

In [4]:
df_train.head(5)

In [5]:
# import seaborn as sns 

# sns.heatmap(df_train.isnull())

**Создание столбца 'is_econom'**

С обозначением типа билета, чтобы сохранить какой билет относится к бизнес/эконом классу.

In [6]:
is_econom = [0] * business_df.shape[0] + [1] * economy_df.shape[0]
df_train['is_econom'] = is_econom

**Удаление дубликатов**

In [7]:
df_train.columns

In [8]:
df_train.duplicated().value_counts()

Существуют 2 дубликата.
Попробуем найти дубликаты по отдельным фичам (исключая целевую - 'price').

In [9]:
df_train.duplicated(subset = ['date', 'airline', 'ch_code', 'num_code', 'dep_time', 'from',
       'time_taken', 'stop', 'arr_time', 'to', 'is_econom']).value_counts()

Из значения True делаем вывод, что на самом деле существует 1479 дубликатов (идентичных друг другу строк с разницей лишь в цене, что может в будущем снизить точность модели).

In [10]:
prev_count = df_train.shape[0]
df_train = df_train.drop_duplicates(subset = ['date', 'airline', 'ch_code', 'num_code', 'dep_time', 'from',
       'time_taken', 'stop', 'arr_time', 'to', 'is_econom'])
print('Всего было: ', prev_count - df_train.shape[0], ' дубликатов')

**Обработка 'stop'**

Так как только в некоторых строках указаны места пересадки, информация является местами нерелевантной, и мы можем сократить значения до количества проделанных остановок.

In [11]:
# df_train.stop.value_counts()

In [12]:
# clear_stop = []
# for stop in df_train.stop:
#     if stop[0] == '1':
#         clear_stop.append('1-stop')
#     elif stop[0] == '2':
#         clear_stop.append('2+-stop')
#     else:
#         clear_stop.append('0-stop')

clear_stop = df_train.stop.str.strip().str[:8].str.strip()
df_train['stop'] = clear_stop

In [13]:
df_train.info()

**Обработка цены**

Необходимо привести колонку 'price' к числовому виду для последующей обработки.

In [14]:
df_train.price.min()

Есть числа с несколькими запятыми, поэтому лучше заменять пропуски на пробелы.

In [15]:
# prices = []
# for price in df_train.price:
#     price = price.replace(',', '')
#     prices.append(int(price))

prices = df_train.price.str.replace(',', '').astype(int).values
df_train['price'] = prices

In [16]:
df_train.head(3)

**Обработка продолжительности полета**

Необходимо обработать нынешний формат продолжительности полета '__h __m' для удобства последующего исследования выборки.

In [17]:
# 1 способ
# time_taken = []
# for period in df_train.time_taken:
#     a, b = period.split()
#     a = a[:-1]
#     if b == '': 
#         b = 0
#     else:
#         b = b[:-1]
#     time_taken.append(float(a) + float(b))

# 2 способ
# def str_to_time(str_time):
#     a, b = list(map(lambda x: x[:-1], str_time.split()))
#     if b: 
#         return int(a) * 60 + int(b)
#     return float(a) * 60
# taken_time_val = df_train.time_taken.apply(str_to_time).values

In [18]:
# 3 способ
hour_values = df_train.time_taken.str.extract('(\d\d)h', expand = False).astype(float) * 60
minute_values = df_train.time_taken.str.extract('(\d\d)m', expand = False).astype(float)
time_taken_in_minutes = hour_values.add(minute_values).astype(float)
df_train['time_taken_minutes'] = time_taken_in_minutes

**Удаление ненужных фичей**

* 'time_taken' - удаляем, т.к. создали новый столбец с длительностью полета в минутах.
* 'arr_time' - удаляем, т.к. она напрямую коррелирует со столбцами времени вылета и длительности общего времени в пути.

In [19]:
df_train['num_code'].value_counts()

'num_code' - удаляем, т.к. столбец имеет слишком много (1255) уникальных значений.

In [20]:
df_train['ch_code'].value_counts()

'ch_code' - удаляем, т.к. столбец имеет несбалансированные классы, что может уменьшить точность предсказания модели (127859 в сравнении с 41).

In [21]:
df_train.columns

In [22]:
df_main = df_train.drop(['time_taken', 'num_code', 'arr_time', 'ch_code'], axis = 1)

**Сохранение данных**

In [23]:
df_main.to_csv('1_features.csv', index = False)

# Сессия 2. Feature Engineering

- Координаты городов. 
- Перелеты внутри государства (расстояние между городами). Скорее всего цена зависит от расхода топлива -> от расстояния
- Дата: выходной.
- Скорость: расстояние / время полета.
- Часто встречаемые авиалинии.
- Время полета (день, утро, вечер).
- В какую минуту дня самолет вылетел.

In [24]:
# Чтение столбцов типа date напрямую из файла csv
# df_main = pd.read_csv('1_features.csv', parse_dates = ['date'], sep = ',')

In [25]:
df_main.date = pd.to_datetime(df_main['date'], format = '%d-%m-%Y')

**Координаты городов**

Находим координаты городов для нахождения расстояния, пройденного за всю поездку. 

In [26]:
!pip install mapbox

[Ссылка](https://docs.mapbox.com/api/search/geocoding/) на документацию Geocoder.

[Ссылка](https://account.mapbox.com/access-tokens) на access_token.

In [27]:
from mapbox import Geocoder

In [28]:
access_token = 'pk.eyJ1IjoiZGFhbmFlYSIsImEiOiJjbDFhNmtiMmgwM3B0M2JwOHYxdGRsa3Q3In0.bPmE3bs4R79Qz9IATniUuQ'
geocoder = Geocoder(access_token = access_token)

In [29]:
all_cities = df_main['from'].append(df_main['to']).unique()

In [30]:
city_coord = {}
for city_name in all_cities:
    response = geocoder.forward(city_name)
    
    place_name = response.json()['features'][0]['place_name']
    coords = response.json()['features'][0]['geometry']['coordinates']
    
    city_coord[city_name] = coords

In [31]:
city_coord

Все перелеты осуществлялись между городами, находящимися в Индии.

In [32]:
from_coord_lon = [] # долгота
from_coord_lat = [] # широта

for city_name in df_main['from']:
    from_coord_lon.append(city_coord[city_name][0])
    from_coord_lat.append(city_coord[city_name][1])

In [33]:
to_coord_lon = []
to_coord_lat = []

for city_name in df_main['to']:
    to_coord_lon.append(city_coord[city_name][0])
    to_coord_lat.append(city_coord[city_name][1])

In [34]:
# Добавляем в датасет
df_main['from_lon'] = from_coord_lon
df_main['from_lat'] = from_coord_lat
df_main['to_lon'] = to_coord_lon
df_main['to_lat'] = to_coord_lat

**Расстояние между городами**

Гипотеза: Цена зависит от расстояния, преодоленного самолетом за всю поездку.

In [35]:
import haversine

In [36]:
distance_val = []
for i in range(len(from_coord_lon)):
    from_lon = from_coord_lon[i]
    from_lat = from_coord_lat[i]
    
    to_lon = to_coord_lon[i]
    to_lat = to_coord_lat[i]
    
    distance = haversine.haversine((from_lat, from_lon), (to_lat, to_lon))
    distance_val.append(distance)

In [37]:
df_main['distance'] = distance_val

**Скорость полета в (км/сек)**

Гипотеза: Цена зависит от скорости полета (чем быстрее => тем дороже). 

In [38]:
df_main['speed'] = df_main['distance'] / (df_main['time_taken_minutes'] / 60)

**День недели**

Предположим существование зависимости между ценой билета и днем недели. 

Гипотеза: В выходные дни цена дороже.

In [39]:
# Добавляем колонку с номерами дней недели 0-6
df_main['weekday'] = df_main['date'].dt.dayofweek

**Время вылета**

- Определить часть дня: день, утро, вечер.
- В какую минуту дня самолет вылетел.

Гипотеза: Существует зависимость между ценой билета и временем вылета .

In [40]:
df_main['dep_minutes'] = df_main.dep_time.apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))

In [41]:
part_of_day_val = []
for time in df_main['dep_minutes']:
    time = float(time / 60)
    if time > 4.5 and time <= 10.5:
        part_of_day_val.append('morning')
    elif time > 10.5 and time <= 16.5:
        part_of_day_val.append('day')
    elif time > 16.5 and time <= 22.5:
        part_of_day_val.append('evening')
    else:
        part_of_day_val.append('night')

In [42]:
df_main['dep_daytime'] = part_of_day_val

**Количество остановок**

Форматируем количество остановок согласно типам, которые были присуждены во время первой сессии, так как это упростить задачу модели при предсказывании цены в будущем.

In [43]:
df_main.stop.value_counts()

In [44]:
df_main['stop_count'] = df_main.stop.map({'1-stop': 1, '2+-stop': 2, 'non-stop': 0})

**Обработка названий авиалиний**

Между уникальными значениями строк фичи существует большой дисбаланс (41 к 127 859), поэтому выделяем три самых распространенных типа, а сотальным присуждаем значение 'Other'.

In [45]:
df_main.airline.value_counts()

In [46]:
airline_val = []
for airline in df_main['airline']:
    if airline != 'Vistara' and airline != 'Air India' and airline != 'Indigo':
        airline_val.append('Other')
    else:
        airline_val.append(airline)

In [47]:
df_main['new_airline'] = airline_val

In [48]:
# Удаляем обработанные столбцы
df_main = df_main.drop(['airline', 'dep_time', 'from', 'to'], axis = 1)

**Погода**

Выборка дана из числа полетов, совершенных в период с 2022-02-11 по 2022-03-31. 
Исходя из этих данных, можно добавить отдельную фичу с погодой в день когда самолет вылетал  (отдельно для каждой даты и города отбытия).

**Сохранение данных**

In [49]:
df_main.to_csv('2_features.csv', index = False)

# Сессия 2. Визуализация данных

Зависимости между разными фичами + корреляция+поиск выбросов.

In [50]:
import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
sns.heatmap(df_main.corr(), annot = True)

**Стоимость билета в зависимости от времени вылета**

In [52]:
group1 = df_main.groupby('dep_daytime').price.mean()
sns.pointplot(group1.index, group1.values)
plt.ylabel('Стоимость билета')
plt.show()

Билеты, чьи вылеты осуществлялись ночью, стоили в среднем гораздо ниже остальных.

**Стоимость билета в зависимости от длительности полета**

In [53]:
group2 = df_main.groupby('time_taken_minutes').price.mean()
sns.pointplot(group2.index, group2.values)
plt.ylabel('Стоимость билета')
plt.show()

**Стоимость билета в зависимости от дальности полета**

In [54]:
group3 = df_main.groupby('distance').price.mean()
sns.pointplot(group3.index.astype(int), group3.values)
plt.ylabel('Стоимость билета')
plt.show()

**Стоимость билета в зависимости от количества пересадок**

In [55]:
group4 = df_main.groupby('stop_count').price.mean()
sns.pointplot(group4.index, group4.values)
plt.ylabel('Стоимость билета')
plt.show()

Билеты за полеты с одной пересадкой стоили дороже с двумя или более пересадками.

При попытке визуализации возникла ошибка 'cannot reindex from a duplicate axis', чтобы ее решить, попробуем найти скрытые дубликаты.

### **Поиск дубликатов**

In [56]:
# Общий поиск дубликатов по выборке
df_main.duplicated().sum()

Существуют 73 дубликатов в строках, которые необходимо удалить.

In [57]:
# Выведем строки-дубликаты
df_main.loc[df_main.duplicated(),:].head(5)

Из таблицы видно, что после изменения значений 'new_airline' на три вида (Vistara, Indigo, Air India и Other), появились дубликаты в данных (с разницей в значениях лишь одной колонки)

In [58]:
df_main.columns

In [59]:
# Поиск дубликатов по всем фичам кроме целевой ('price')
df_main.duplicated(subset=['date', 'stop', 'is_econom', 'time_taken_minutes', 'from_lon',
       'from_lat', 'to_lon', 'to_lat', 'distance', 'speed', 'weekday',
       'dep_minutes', 'dep_daytime', 'stop_count', 'new_airline']).sum()

Существуют еще 574 дубликатов в строках, которые необходимо удалить.

In [60]:
# Создаем новую выборку без дубликатов
df_main = df_main.drop_duplicates(subset=['date', 'stop', 'is_econom', 'time_taken_minutes', 'from_lon',
       'from_lat', 'to_lon', 'to_lat', 'distance', 'speed', 'weekday',
       'dep_minutes', 'dep_daytime', 'stop_count', 'new_airline'])

In [61]:
# Поиск дубликатов в строках с учетом корреляции между фичами
df_main.duplicated(subset=['date', 'stop', 'is_econom', 'from_lon',
       'from_lat', 'to_lon', 'to_lat', 'speed','dep_daytime', 'stop_count', 'new_airline']).sum()

In [62]:
df_main = df_main.drop_duplicates(subset = ['date', 'stop', 'is_econom', 'from_lon',
       'from_lat', 'to_lon', 'to_lat', 'speed','dep_daytime', 'stop_count', 'new_airline'])

In [63]:
df_main.info()

In [64]:
df_main.index.duplicated()

In [65]:
df = df_main

In [66]:
df.columns

In [67]:
df.index.duplicated().any()

Выводит True, значит в выборке есть дубликаты.

In [68]:
df.loc[~df.index.duplicated(),:].head(5)

In [69]:
# Сохраняем первое встречающееся значение из нескольких дубликатов
df = df[~df.index.duplicated(keep='first')]

### Поиск выбросов

Используя функцию распределения (фокус на края).

Графики для разных типов данных:
* количественные;
* категориальные - barplot, scatterplot, boxplot (при норм. распределении);
* вещественные - histplot, scatterplot;
* порядковые данные.

boxplot - нормальное распределение 
0.25/0.75 квантилей верх/вниз

In [70]:
sns.scatterplot( 
    data = df_main, 
    x = 'speed', 
    y = 'price',
    hue = 'new_airline',
    alpha = 0.5
)

Из графика видно, что зависимости между скоростью и авиалинией нет. Но стоит учитывать пересадки, проделанные во время одной поездки.

**Зависимость цены от типа билета ('is_econom')**

In [71]:
sns.histplot(data = df, x = 'price', hue = 'is_econom')

Из графика можем проследить распределение Пуассона.

**Зависимость скорости от количества проделанных остановок**

In [72]:
sns.histplot(data = df, x = 'speed', hue = 'stop_count')

In [73]:
df.speed.max()

Современные авиалайнеры легко развивают скорость в 500 км/ч. Но и эта цифра не является пределом возможностей самолетов. Оптимальный средний показатель скорости, это *800 км/ч* (что указывает на отсутствие выбросов).

**Зависимость времени в пути от количества проделанных остановок**

In [74]:
sns.histplot(data = df, x = 'time_taken_minutes', hue = 'stop_count')

**Зависимость цены от скорости**

In [75]:
df['type_of_fly'] = df['stop_count'].astype('str') + df['is_econom'].astype('str')

In [76]:
sns.scatterplot(
    data = df,
    x = 'speed',
    y = 'price',
    hue = 'type_of_fly',
    alpha = 0.5
)

**Зависимость цены от количества пересадок**

In [77]:
sns.barplot(data = df, x = 'stop_count', y = 'price', hue = 'is_econom')

Из графика видно, что чем больше пересадок => тем больше цена (возможно из-за дальности полета).

In [78]:
df.describe()

**Обзор цены**

In [79]:
df.price.max()

In [80]:
df.price.min()

**Обзор длительности полета**

In [81]:
sns.scatterplot(
    data = df,
    x = 'time_taken_minutes',
    y = 'price',
    hue = 'stop_count',
    alpha = 0.5
)

### **Кластеризация**

In [82]:
# Группирование по координатам from
plt.scatter(df['from_lat'], df['from_lon'])
# Группирование по координатам to
plt.scatter(df['to_lat'], df['to_lon'])

6 точек отбытия и прибытия. 

In [83]:
from sklearn.cluster import KMeans 
knn = KMeans(n_clusters = 6)

from_coords = df[['from_lat', 'from_lon']]
to_coords = df[['to_lat', 'to_lon']]

df['from_cluster'] = knn.fit(from_coords).labels_
df['to_cluster'] = knn.fit(to_coords).labels_

In [84]:
sns.scatterplot(data = df, x = 'from_lat', y = 'from_lon')
sns.scatterplot(data = df, x = 'to_lat', y = 'to_lon')

In [85]:
df.head(5)

In [86]:
df.info()

In [87]:
# Удаляем строки с пустыми значениями
df = df.dropna()

### **Разделение на обучающую/тестирующую выборку**

Для последующего обучения.

In [88]:
drop_col = ['date', 'stop', 'price', 'from_lon',
       'from_lat', 'to_lon', 'to_lat', 'speed', 
       'dep_minutes', 'type_of_fly', 'from_cluster', 'to_cluster']

In [91]:
from sklearn.model_selection import train_test_split

train = df.drop(drop_col, axis=1)
target = df.price
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

# Сессия 3. Обучение модели

In [169]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

### **Feature Transformation**

Трансформация данных для повышения точности модели:
* Кодирование категориальных колонок (OHE, LE).
* Нормализация вещественных колонок (MMS, SS).
* Заполнение пустых ячеек.

In [93]:
# Разделение колонок на вещественные и категрилаьные
num_cols = ['time_taken_minutes', 'distance']
cat_cols_ohe = ['stop_count', 'is_econom', 'new_airline', 'dep_daytime', 'weekday']
target = 'price' 

In [ ]:
# imputation = ColumnTransformer([
#     ('categ_imputer', SimpleImputer(strategy='most_frequent'), [cat_cols_le+cat_cols_ohe]),
#     ('numer_imputer', SimpleImputer(strategy='mean'), ['time_taken_minutes', 'distance'])
# ])

# imputation.fit_transform(X_train)

In [94]:
ct = ColumnTransformer([
    ('scaler', MinMaxScaler(), num_cols),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols_ohe)
])

### **Построение пайплайна**

**Random Forest** (базовая модель)

In [101]:
base_pipe = Pipeline(steps = [
    ('transformer', ct),
    ('model', RandomForestRegressor())
])

In [102]:
base_pipe.fit(X_train, y_train)

mae_val = 0
mse_val = 0
for i in range(5):
    y_pred = base_pipe.predict(X_test)
    mae_val += mean_absolute_error(y_test, y_pred)
    mse_val += mean_squared_error(y_test, y_pred)

print('Средняя точность модели (MAE): ', mae_val/5)
print('Средняя точность модели (MSE): ', mse_val/5)

Средняя точность базовой модели RandomForestRegressor:

* MAE:  2420.360905132962
* MSE:  17311283.44129897

In [105]:
# Проверка значений предсказанных цен
y_pred = base_pipe.predict(X_test)
print(y_pred.min(), y_pred.max())

Цены имеют положительное значение, что означает что модель работает корректно.

In [106]:
sns.scatterplot(x=y_pred, y=y_test, hue=X_test.is_econom)

**XGBoost**

In [151]:
xgb_pipe = Pipeline([
    ('transformer', ct),
    ('model', XGBRegressor())
])

In [152]:
xgb_pipe.fit(X_train, y_train)

mae_val = 0
mse_val = 0
for i in range(3):
    y_pred = xgb_pipe.predict(X_test)
    mae_val += mean_absolute_error(y_test, y_pred)
    mse_val += mean_squared_error(y_test, y_pred)

print('Средняя точность (МАЕ):', mae_val/3)
print('Средняя точность (MSE):', mse_val/3)

Средняя точность модели XGBoost:

* MAE:  3006.1765688482883
* MSE:  21127817.94761643

Результат хуже, чем у базовой модели RandomForest.

In [163]:
# Проверка значений предсказанных цен
y_pred = xgb_pipe.predict(X_test)
print(y_pred.min(), y_pred.max()) # -1373.3813 89210.48

Модель выводит цены со значением < 0 на некоторые билеты (что невозможно).

Возможно, данная ошибка возникла из-за переобучения модели. Попробуем решить ее с помощью форматирования входных-выходных данных целевого признака ('price'). 

In [154]:
from sklearn.compose import TransformedTargetRegressor

In [155]:
xgb_tt = TransformedTargetRegressor(
    regressor = XGBRegressor(),
    func = np.log,
    inverse_func = np.exp
)

In [156]:
xgb_pipe_with_tt = Pipeline([
    ('transformer', ct),
    ('model', xgb_tt)
])

In [158]:
xgb_pipe_with_tt.fit(X_train, y_train)

mae_val = 0
mse_val = 0
for i in range(3):
    y_pred = xgb_pipe_with_tt.predict(X_test)
    mae_val += mean_absolute_error(y_test, y_pred)
    mse_val += mean_squared_error(y_test, y_pred)

print('Средняя точность (МАЕ):', mae_val/3)
print('Средняя точность (MSE):', mse_val/3)

Средняя точность модели XGBoost:

* MAE:  3080.903076606419
* MSE:  23766442.23357633

Результат немного хуже, чем у модели XGBoost без трансформации целевого признака.

In [159]:
# Проверка значений предсказанных цен
y_pred = xgb_pipe_with_tt.predict(X_test)
print(y_pred.min(), y_pred.max()) # 1156.3638 90373.125

Цены имеют положительное значение, что означает что новая модель XGBoost работает корректно*.

In [165]:
sns.scatterplot(x=y_pred, y=y_test, hue=X_test.is_econom)

**CatBoost**

In [170]:
ctb_pipe = Pipeline([
    ('transformer', ct),
    ('model', CatBoostRegressor())
])

In [173]:
ctb_pipe.fit(X_train, y_train)

mae_val = 0
mse_val = 0
for i in range(5):
    y_pred = ctb_pipe.predict(X_test)
    mae_val += mean_absolute_error(y_test, y_pred)
    mse_val += mean_squared_error(y_test, y_pred)

print('Средняя точность модели (MAE): ', mae_val/5)
print('Средняя точность модели (MSE): ', mse_val/5)

Средняя точность модели CatBoost:

* MAE:  3127.3894443914364
* MSE:  22399653.98018688

Это худший результат среди трех представленных выше.

In [172]:
# Проверка значений предсказанных цен
y_pred = ctb_pipe.predict(X_test)
print(y_pred.min(), y_pred.max()) # 1800.45451897151 87293.80046527024

Цены имеют положительное значение, что означает что модель работает корректно.

### **Feature Selection (Importances)**

Отсечение ненужных признаков.

**Random Forest**

In [148]:
def get_rfr_feature_importances(forest, feature_names):
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    forest_importances = pd.Series(importances, index=feature_names)
    
    fig, ax = plt.subplots()
    forest_importances.plot.bar(yerr=std, ax=ax)
    ax.set_title('Важность отдельных признаков')
    ax.set_ylabel('Среднее значение точности')
    fig.show()

In [109]:
base_pipe.named_steps['model']

In [139]:
X_train.weekday.sort_values().unique()

In [136]:
X_train[cat_cols_ohe]

In [183]:
feature_names = num_cols + ['0_stop', '1_stop', '2+_stops'] + ['0_is_econom', '1_is_econom'] + ['Air India', 'Indigo', 'Other', 'Vistara'] + ['day', 'evening', 'morning', 'night'] + ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

In [184]:
get_rfr_feature_importances(
    base_pipe.named_steps['model'],
    feature_names
)

Из графика видно, что главными признаками для модели Random Forest являются класс билета ('is_econom'), время полета и значение расстояния, преодоленного за всю поездку. Интересно, что незначительное влияние оказывает принадлежность самолета к компании Vistara/Air India.

**XGBoost**

In [208]:
feature_importances.keys()

In [186]:
# Берем все значения за исключением третьей и седьмой колонок, т.к. они не играют роли
feature_names = num_cols + ['1_stop', '2+_stops'] + ['0_is_econom'] + ['Air India', 'Indigo', 'Other', 'Vistara'] + ['day', 'evening', 'morning', 'night'] + ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

In [210]:
feature_importances = xgb_pipe.named_steps['model'].get_booster().get_score(importance_type='weight')
keys = list(feature_names)
values = list(feature_importances.values())

data = pd.DataFrame(data=values, index=feature_names, columns=['price']).sort_values(by='price', ascending=False)
data.nlargest(20, columns='price').plot(kind='barh', figsize = (20,10)) # топ 20 признаков

Главными признаками для модели XGBoost являются время полета, значение расстояния, преодоленного за всю поездку, и класс билета ('is_econom') - так же, как для Random Forest. Однако примечательно, что точность модели зависит от большего количества признаков - авиалиний, дня недели, времени вылета и количества остановок.

**CatBoost**

Ссылка на [источник](https://towardsdatascience.com/deep-dive-into-catboost-functionalities-for-model-interpretation-7cdef669aeed#:~:text=To%20get%20this%20feature%20importance,all%20the%20trees%20in%20the).

In [215]:
import shap
from sklearn import metrics

In [218]:
# shap_values = ctb_pipe.named_steps['model'].get_feature_importance(Pool(X_test, label=y_test,cat_features=categorical_features_indices),type="ShapValues")
# expected_value = shap_values[0,-1]
# shap_values = shap_values[:,:-1]

# shap.initjs()
# shap.force_plot(expected_value, shap_values[3,:], X_test.iloc[3,:])

### **Оптимизация**